Link: https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/3_NeuralNetworks/convolutional_network_raw.py

In [67]:
import tensorflow as tf

import os

import numpy as np

from alexnet import AlexNet
from datetime import datetime
from tensorflow.contrib.data import Iterator

from datagenerator import ImageDataGenerator
from tensorflow.contrib.data import Iterator



In [45]:
# Training Parameters
learning_rate = 0.001
num_steps = 100
batch_size = 32
display_step = 10

# Network params
dropout_rate = 0.5
num_classes = 2

# How often we want to write the tf.summary data to disk
display_step = 20


In [ ]:
import csv
import pickle
import random

dirroot='../dataset_patches/'
leveldir='/level1/'

stagecsv='../../data-wsi/camelyon17/stage_labels.csv'
anno_path='../../data-wsi/camelyon17/lesion_annotations_new/'

positive_annoAll=list()

positive_anno=list()
positive_none=list()

negative_slides=list()

posanno=0
posnone=0
neg=0

labeldict=dict()


print('total number of annotation: ', len(os.listdir(anno_path)))

a=0
for i in [i for i in os.listdir(anno_path)[:]]:
    a+=1
    slidename=i.split('.')[0]
    positive_annoAll.append(slidename)

with open(stagecsv) as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if '.tif' in row['patient']:
            labeldict[row['patient'].split('.')[0]] = row['stage']
            

for slidedir in os.listdir(dirroot):
    
    if slidedir in list(labeldict.keys()):
        '''
        '''
        slidepath=dirroot+slidedir+leveldir
        
        if labeldict[slidedir] == 'negative':
            '''
                negative
            '''
            neg += 1
            negative_slides.append(slidepath)
            
        else:
            if slidedir in positive_annoAll:
                '''
                    annotated positive
                '''
                posanno += 1
                positive_anno.append(slidepath)
                
            else:
                '''
                    positive none
                '''
                posnone += 1
                positive_none.append(slidepath)

print("negative:", neg, len(negative_slides))

print("positive anno:", posanno, len(positive_anno))
print("positive none:", posnone, len(positive_none))

negset=list()

for negpath in negative_slides:
    with open(negpath + 'negpaths.txt', "rb") as fp:   # Unpickling
        neg_path_tmp = pickle.load(fp)
    negset += neg_path_tmp

print('neg patches in neg slides:', len(negset))

posAnnoSet=list()
negAnnoSet=list()

for path in positive_anno:
    
    if not os.path.isfile(path + 'pospaths.txt'):
        pass
    
    else:
        with open(path + 'negpaths.txt', "rb") as fp:   # Unpickling
            negpath_tmp = pickle.load(fp)
        negAnnoSet += negpath_tmp

        with open(path + 'pospaths.txt', "rb") as fp:   # Unpickling
            pospath_tmp = pickle.load(fp)
        posAnnoSet += pospath_tmp
        
print('neg patches in neg slides:', len(negAnnoSet))
print('pos patches in pos slides:', len(posAnnoSet))

num_to_select = 5000
group_of_items = posAnnoSet
pospatches = random.sample(group_of_items, num_to_select)

num_to_select = 5000
group_of_items = negAnnoSet
negpatches = random.sample(group_of_items, num_to_select)

num_to_select = 500
group_of_items = posAnnoSet
pospatchesval = random.sample(group_of_items, num_to_select)

num_to_select = 500
group_of_items = negAnnoSet
negpatchesval = random.sample(group_of_items, num_to_select)


with open('../0506samples.txt', "w") as fp:
    for item in pospatches:
        fp.write(item + ' 1\n')

    for item in negpatches:
        fp.write(item + ' 0\n')

with open('../0506samplesval.txt', "w") as fp:
    for item in pospatchesval:
        fp.write(item + ' 1\n')

    for item in negpatchesval:
        fp.write(item + ' 0\n')

In [11]:
filewriter_path = '/tmp/finetune_alexnet/tensorboard'
checkpoint_path = '/tmp/finetune_alexnet/checkpoints'

train_file = '../0506samples.txt'
val_file = '../0506samplesval.txt'

assert(os.path.isfile(train_file))
assert(os.path.isfile(val_file))

In [12]:
if not os.path.isdir(checkpoint_path):
    os.mkdir(checkpoint_path)

In [44]:
with tf.device('/cpu:0'):
    tr_data = ImageDataGenerator(train_file,
                                 mode='training',
                                 batch_size=batch_size,
                                 num_classes=num_classes,
                                 shuffle=True)
    val_data = ImageDataGenerator(val_file,
                                  mode='inference',
                                  batch_size=batch_size,
                                  num_classes=num_classes,
                                  shuffle=False)

    # create an reinitializable iterator given the dataset structure
    iterator = Iterator.from_structure(tr_data.data.output_types,
                                       tr_data.data.output_shapes)
    next_batch = iterator.get_next()

In [48]:
# Ops for initializing the two different iterators
training_init_op = iterator.make_initializer(tr_data.data)
validation_init_op = iterator.make_initializer(val_data.data)

# TF placeholder for graph input and output
x = tf.placeholder(tf.float32, [batch_size, 227, 227, 3])
y = tf.placeholder(tf.float32, [batch_size, num_classes])
keep_prob = tf.placeholder(tf.float32)

# Initialize model
model = AlexNet(x, keep_prob, num_classes, skip_layer=None)

# Link variable to model output
score = model.fc8

In [58]:
# List of trainable variables of the layers we want to train
var_list = [v for v in tf.trainable_variables()]

In [52]:
# Op for calculating the loss
with tf.name_scope("cross_ent"):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=score,
                                                                  labels=y))


In [53]:
# Train op
with tf.name_scope("train"):
    # Get gradients of all trainable variables
    gradients = tf.gradients(loss, var_list)
    gradients = list(zip(gradients, var_list))

    # Create optimizer and apply gradient descent to the trainable variables
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train_op = optimizer.apply_gradients(grads_and_vars=gradients)


In [64]:
# Add gradients to summary
for gradient, var in gradients:
    tf.summary.histogram(var.name.replace(":", "_") + '/gradient', gradient)

# Add the variables we train to the summary
for var in var_list:
    tf.summary.histogram(var.name.replace(":", "_"), var)

# Add the loss to summary
tf.summary.scalar('cross_entropy', loss)

# Evaluation op: Accuracy of the model
with tf.name_scope("accuracy"):
    correct_pred = tf.equal(tf.argmax(score, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Add the accuracy to the summary
tf.summary.scalar('accuracy', accuracy)

# Merge all summaries together
merged_summary = tf.summary.merge_all()

# Initialize the FileWriter
writer = tf.summary.FileWriter(filewriter_path)



In [65]:
# Initialize an saver for store model checkpoints
saver = tf.train.Saver()

In [68]:
# Get the number of training/validation steps per epoch
train_batches_per_epoch = int(np.floor(tr_data.data_size/batch_size))
val_batches_per_epoch = int(np.floor(val_data.data_size / batch_size))

print(train_batches_per_epoch, val_batches_per_epoch)


312 31


In [69]:
# Start Tensorflow session
with tf.Session() as sess:

    # Initialize all variables
    sess.run(tf.global_variables_initializer())

    # Add the model graph to TensorBoard
    writer.add_graph(sess.graph)

    # Load the pretrained weights into the non-trainable layer
    model.load_initial_weights(sess)

    print("{} Start training...".format(datetime.now()))
    print("{} Open Tensorboard at --logdir {}".format(datetime.now(),
                                                      filewriter_path))

    # Loop over number of epochs
    for epoch in range(num_epochs):

        print("{} Epoch number: {}".format(datetime.now(), epoch+1))

        # Initialize iterator with the training dataset
        sess.run(training_init_op)

        for step in range(train_batches_per_epoch):

            # get next batch of data
            
            st=time.time()
            img_batch, label_batch = sess.run(next_batch)
            et=time.time()
            
            print('fetching batch time: %.3f' % et - st)
            
            st1=time.time()
            # And run the training op
            sess.run(train_op, feed_dict={x: img_batch,
                                          y: label_batch,
                                          keep_prob: dropout_rate})
            et1=time.time()
            print('training time: %.3f' % et1 - st1)
            
            # Generate summary with the current batch of data and write to file
            if step % display_step == 0:
                s = sess.run(merged_summary, feed_dict={x: img_batch,
                                                        y: label_batch,
                                                        keep_prob: 1.})

                writer.add_summary(s, epoch*train_batches_per_epoch + step)

        # Validate the model on the entire validation set
        print("{} Start validation".format(datetime.now()))
        sess.run(validation_init_op)
        test_acc = 0.
        test_count = 0
        for _ in range(val_batches_per_epoch):

            img_batch, label_batch = sess.run(next_batch)
            acc = sess.run(accuracy, feed_dict={x: img_batch,
                                                y: label_batch,
                                                keep_prob: 1.})
            test_acc += acc
            test_count += 1
        test_acc /= test_count
        print("{} Validation Accuracy = {:.4f}".format(datetime.now(),
                                                       test_acc))
        print("{} Saving checkpoint of model...".format(datetime.now()))

        # save checkpoint of the model
        checkpoint_name = os.path.join(checkpoint_path,
                                       'model_epoch'+str(epoch+1)+'.ckpt')
        save_path = saver.save(sess, checkpoint_name)

        print("{} Model checkpoint saved at {}".format(datetime.now(),
                                                       checkpoint_name))



FileNotFoundError: [Errno 2] No such file or directory: 'bvlc_alexnet.npy'